In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/AIMO3_Reference_Problems.pdf
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/aimo_3_inference_server.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/aimo_3_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/__init__.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/core/templates.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/core/base_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/core/relay.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/ai-mathematical-olympiad-progre

In [2]:
# ===================================================================
# FINAL SUBMISSION BUILD SCRIPT v2.1
# STRATEGY: BUILD THE AGENT. DO NOT RUN THE RACE.
# This script's only job is to generate 'submission.tar.gz'.
# ===================================================================

import os
import tarfile

print("▶ [BUILD] Initiating 'Operation: Trojan Horse' (v2.1)...")
print("   > STRATEGY: Building the agent, not running the race.")

# --- STEP 1: Create the directory for our agent ---
if not os.path.exists('kaggle_submission'):
    os.makedirs('kaggle_submission')
print("   > Secure directory 'kaggle_submission' created.")

# --- STEP 2: Write the requirements file ---
requirements_content = """
sympy
numpy
"""
with open('kaggle_submission/requirements.txt', 'w') as f:
    f.write(requirements_content)
print("   > Dependency file 'requirements.txt' written.")

# --- STEP 3: Write the main agent script (The Prometheus Engine) ---
main_script_content = """
# main.py - The core script for our AI agent.
# Kaggle's servers will execute this file in their environment.

import re
import math
import random
import time
import numpy as np

# This block contains the perfected Prometheus Engine.
try:
    from sympy import isprime, factorint
    from sympy.ntheory.modular import crt
except ImportError:
    def isprime(n):
        if n <= 1: return False
        if n > 2 and n % 2 == 0: return False
        for i in range(3, int(math.sqrt(n)) + 1, 2):
            if n % i == 0: return False
        return True
    # Simplified fallbacks for safety
    def factorint(n):
        factors = {}
        d = 2
        while d * d <= n:
            while n % d == 0:
                factors[d] = factors.get(d, 0) + 1
                n //= d
            d += 1
        if n > 1: factors[n] = factors.get(n, 0) + 1
        return factors
    def crt(m, v):
        N = np.prod(m)
        result = 0
        for mi, vi in zip(m, v):
            Ni = N // mi
            result += vi * pow(Ni, -1, mi) * Ni
        return result % N, N

def get_largest_prime_factor(n):
    if n <= 1: return 1
    d = 2
    largest_factor = 1
    while d * d <= n:
        if n % d == 0:
            largest_factor = d
            while n % d == 0:
                n //= d
        d += 1
    if n > 1:
        largest_factor = n
    return largest_factor

class NeoCerberus:
    def solve(self, problem_text):
        try:
            matches = re.findall(r"remainder (?:of )?(\d+) when divided by (\d+)", problem_text, re.IGNORECASE)
            if len(matches) >= 2:
                moduli = [int(m[1]) for m in matches]
                remainders = [int(m[0]) for m in matches]
                solution, _ = crt(moduli, remainders)
                return solution
        except:
            return None
        return None

def sympy_solver(problem_text):
    text = problem_text.lower()
    try:
        if "sum of all integers" in text and "multiple of" in text:
            nums = [int(s) for s in re.findall(r'\\b\\d+\\b', text)]
            nums = [n for n in nums if n > 1]
            if len(nums) >= 2:
                limit, k = max(nums), min(nums)
                n_terms = limit // k
                return k * (n_terms * (n_terms + 1)) // 2
        if "trailing zeros" in text and "!" in text:
            num = int(re.search(r'(\\d+)!', text).group(1))
            count = 0
            i = 5
            while (num // i >= 1):
                count += num // i
                i *= 5
            return count
    except:
        return None
    return None

class PrometheusSentinel:
    def __init__(self):
        self.neo = NeoCerberus()

    def solve(self, problem_text):
        geometry_keywords = ["triangle", "circle", "angle", "geometry", "points", "plane", "coordinate"]
        if any(kw in problem_text.lower() for kw in geometry_keywords):
            return None

        # Router Logic
        res = self.neo.solve(problem_text)
        if res is not None: return res
        
        if "largest prime factor" in problem_text.lower():
            numbers = [int(s) for s in re.findall(r'\\b\\d+\\b', text)]
            if numbers: return get_largest_prime_factor(max(numbers))
            
        res = sympy_solver(problem_text)
        if res is not None: return res
        
        return None

# The AIMOAgent is the public interface for the Kaggle judge.
class AIMOAgent:
    def __init__(self):
        self.engine = PrometheusSentinel()
        print("[PROMETHEUS] Live Agent Initialized. Doctrine: Number Theory Dominance.")

    def solve_problem(self, problem_text):
        answer = self.engine.solve(problem_text)
        return int(answer) if answer is not None else 0

if __name__ == "__main__":
    # This block is for the Kaggle judge's execution environment.
    # It will import this main.py and use the AIMOAgent class.
    agent = AIMOAgent()
    print("[PROMETHEUS] System ready. Standing by for evaluation.")
"""
with open('kaggle_submission/main.py', 'w') as f:
    f.write(main_script_content)
print("   > Core logic 'main.py' written.")

# --- STEP 4: Package the agent into the submittable .tar.gz archive ---
with tarfile.open('submission.tar.gz', 'w:gz') as tar:
    tar.add('kaggle_submission', arcname='.')
print("   > Agent successfully packaged into 'submission.tar.gz'.")

print("\n" + "-" * 60)
print("✅ 'Operation: Trojan Horse' COMPLETE.")
print("▶ You MUST submit the file named 'submission.tar.gz'.")
print("-" * 60)



▶ [BUILD] Initiating 'Operation: Trojan Horse' (v2.1)...
   > STRATEGY: Building the agent, not running the race.
   > Secure directory 'kaggle_submission' created.
   > Dependency file 'requirements.txt' written.
   > Core logic 'main.py' written.
   > Agent successfully packaged into 'submission.tar.gz'.

------------------------------------------------------------
✅ 'Operation: Trojan Horse' COMPLETE.
▶ You MUST submit the file named 'submission.tar.gz'.
------------------------------------------------------------
